In [50]:
# Library import
import math
import numpy as np
import pandas as pd
from IPython.display import Image
import re

# Clarke Network Based Distance Protection
## Boundary Conditions For Fault Types
### Single Phase Faults
\begin{equation} \tilde{V}_a = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_b = \Delta \tilde{I}_c = 0 \end{equation}
\begin{equation} \Delta \tilde{V}_\alpha + \Delta \tilde{V}_0 = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_\beta = 0 \end{equation}
\begin{equation} k_0 \times \Delta \tilde{I}_\alpha = \Delta \tilde{I}_0 = 0 \end{equation}

### Three Phase Faults
\begin{equation} \tilde{V}_a = \tilde{V}_b = \tilde{V}_c = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_a + \Delta \tilde{I}_b + \Delta \tilde{I}_c = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_\alpha = 0 \end{equation}
\begin{equation} \Delta \tilde{V}_\alpha = 0 \end{equation}
\begin{equation} \Delta \tilde{V}_\beta = 0 \end{equation}

### Phase to Phase Faults (BC Fault)
\begin{equation} \tilde{I}_a = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_b + \Delta \tilde{I}_c = 0 \end{equation}
\begin{equation} \Delta \tilde{V}_b = \Delta \tilde{V}_c = 0 \end{equation}
\begin{equation} \Delta \tilde{I}_\alpha = \Delta \tilde{I}_0 = 0 \end{equation}
\begin{equation} \Delta \tilde{V}_\beta = 0 \end{equation}

### Logic Condition to Fix the Numerical Error in Calculation k0
>>>if: 
\begin{equation} \bigg( \bigg |\frac{i_0(k)}{i_\alpha(k)}\bigg| > 0.5 \bigg) || \Bigg( \Bigg| \frac{\big|\frac{i_0(k)}{i_\alpha (k)} \big| - \big|\frac{i_0(k-1)}{i_\alpha (k-1)} \big|}{\big|\frac{i_0(k-1)}{i_\alpha (k-1)} \big|} \Bigg|\times100 \Bigg) > k\end{equation} 
>>>then:
\begin{equation} \big|\frac{i_0(k)}{i_\alpha (k)} \big| = \big|\frac{i_0(k-1)}{i_\alpha (k-1)} \big| \end{equation}

### Fault Resistance Model (Ground Faults)
<img src="fault_resistance_model.png" alt="drawing" width="300"/>
Estimated inductance
\begin{equation} L_{est} = x \times (L_1 + k_0 L_0) \end{equation}
Estimated Resistance
\begin{equation} R_{est} = x \times (R_1 + k_0 R_0) + kR_f \end{equation}
<br>
Line inductance ratio (known quantity)
<br>
\begin{equation} \frac{L_0}{L_1} = k_1 \end{equation}
<br>
Line resistance ratio (known quantity)
<br>
\begin{equation} \frac{R_0}{R_1} = k_2 \end{equation}
<br>
\begin{equation} x = \frac{L_{est}}{L_1(1+k_0k_1)} \end{equation}
<br>
\begin{equation} R_f = R_{est} - \frac{L_{est}R_1(1+k_2k_0)}{L_1(1 + k1k_0)} \end{equation}
<br>
\begin{equation} R_{Loop} = R_{est} - R_f \end{equation}

In [6]:
Vbase = 230;
Sbase = 100;
Zbase = Vbase**2/Sbase;
print(Zbase)

529.0


In [7]:
# Per unit 
R1 = 0.964957982E-02
X1 = 0.118248408j
R0 = 0.812354658E-01
X0 = 0.325471390j
Z1 = R1+X1
Z0 = R0+X0
L1 = (X1/(2*math.pi*50)).imag
L0 = (X0/(2*math.pi*50)).imag
k1 = L0/L1
k2 = R0/R1

In [8]:
# Source Impedance
Rs = 0.994772
Xs = 49.990j

In [12]:
Z = (Z1+Z0/2)*Zbase
# Print Impedance Values for different length
location = 0
print('R and L estimates in Ground Fault Loop')
print('Location \t R \t\t\t L')
Rest_LG = []
Lest_LG = []
for x in range(1,10):
    print(location+x*10, '\t\t', round((Z*(location+x*10/100)).real,4),'\t','\t',round((Z*(location+x*10/100)).imag/(2*math.pi*50),4))
    Rest_LG += [round((Z*(location+x*10/100)).real,4)]
    Lest_LG += [round((Z*(location+x*10/100)).imag/(2*math.pi*50),4)]
print(Rest_LG)
print(Lest_LG)

R and L estimates in Ground Fault Loop
Location 	 R 			 L
10 		 2.6591 	 	 0.0473
20 		 5.3183 	 	 0.0946
30 		 7.9774 	 	 0.1419
40 		 10.6366 	 	 0.1893
50 		 13.2957 	 	 0.2366
60 		 15.9548 	 	 0.2839
70 		 18.614 	 	 0.3312
80 		 21.2731 	 	 0.3785
90 		 23.9323 	 	 0.4258
[2.6591, 5.3183, 7.9774, 10.6366, 13.2957, 15.9548, 18.614, 21.2731, 23.9323]
[0.0473, 0.0946, 0.1419, 0.1893, 0.2366, 0.2839, 0.3312, 0.3785, 0.4258]


In [13]:
Zph = Z1*Zbase
location = 0
print('R and L estimates in Phase Fault Loop')
print('Location \t R \t\t\t L')
Rest_LL = []
Lest_LL = []
for x in range(1,10):
    print(location+x*10, '\t\t', round((Zph*(location+x*10/100)).real,4),'\t','\t',round((Zph*(location+x*10/100)).imag/(2*math.pi*50),4))
    Rest_LL += [round((Zph*(location+x*10/100)).real,4)]
    Lest_LL += [round((Zph*(location+x*10/100)).imag/(2*math.pi*50),4)]
print(Rest_LL)
print(Lest_LL)

(5.10462772478+62.553407832j)
R and L estimates in Phase Fault Loop
Location 	 R 			 L
10 		 0.5105 	 	 0.0199
20 		 1.0209 	 	 0.0398
30 		 1.5314 	 	 0.0597
40 		 2.0419 	 	 0.0796
50 		 2.5523 	 	 0.0996
60 		 3.0628 	 	 0.1195
70 		 3.5732 	 	 0.1394
80 		 4.0837 	 	 0.1593
90 		 4.5942 	 	 0.1792
[0.5105, 1.0209, 1.5314, 2.0419, 2.5523, 3.0628, 3.5732, 4.0837, 4.5942]
[0.0199, 0.0398, 0.0597, 0.0796, 0.0996, 0.1195, 0.1394, 0.1593, 0.1792]


In [60]:
SheetNames = ['10','20','30','40','50','60','70','80','90']
df_dict = pd.read_excel('C:/Users/wiwmigjs/OneDrive - University of Manitoba (1)/PHD/Research/Incremental/Excel/Clarke Fault Loop Estimation/Second Attempt/Single Ended/5 Ohm/AG.xlsx', sheet_name=None, skiprows=[0])

In [65]:
ialpha_post_fault = 0;
izero_post_fault = 0;
VTR = 2000;
CTR = 200;
dt = 250E-6; # Changed dt value to 50E-6
fs = 1/dt;
f = 50;
T = 1/f;
NSC = T/dt;
Tfault = 0.998;
Ralpha_act = 2.699;
Lalpha_act = 14.956/(2*math.pi*50);
for k in range(0,len(SheetNames)):
    # Get data from
    df = df_dict.get(SheetNames[k])
    t = (df.columns[0])
    print(t)
    #t = t[6:]
    #[float(i) for i in t]
    
    ialpha = df.columns[1]
    ibeta = df.columns[2]
    izero = df.columns[3]
    valpha = df.columns[4]
    vbeta = df.columns[5]
    vzero = df.columns[6]
    idx = 1;
    mat_len = NSC/8;
    mat_idx = 1;
    bufful = 0;
    k = 1;
    Lprev = 0;
    Lest_err = 0;
    trip_cnt = 0;
    Trip = np.zeros((9,600));
    i = 0
    #while i < len(t)-1:
        #if t[i] >= (Tfault):
           # print('Fault')
        #i = i+1
        

# Preview
#print(len(df_feb.columns))
#print(df_feb.iloc[:, 1])

12.7430997308
12.7430814481
12.7430866492
12.7430709539
12.7432329946
12.7430589326
12.7430626405
12.7430454711
12.7430518432
